In [ ]:
#FUNCTIONS TAKEN FROM LECTURE NOTES TO BUILD UPON
import math

class MinHeap:
    def __init__(self, data):
        self.items = data
        self.length = len(data)
        self.build_heap()
        # add a map based on input node
        self.map = {}
        for i in range(self.length):
            self.map[self.items[i].value] = i

    def find_left_index(self,index):
        return 2 * (index + 1) - 1

    def find_right_index(self,index):
        return 2 * (index + 1)

    def find_parent_index(self,index):
        return (index + 1) // 2 - 1  

    def sink_down(self, index):
        smallest_known_index = index

        left_index = self.find_left_index(index)
        right_index = self.find_right_index(index)
        if left_index < self.length and self.items[left_index].key < self.items[smallest_known_index].key:
            smallest_known_index = left_index

        if right_index < self.length and self.items[right_index].key < self.items[smallest_known_index].key:
            smallest_known_index = right_index

        if smallest_known_index != index:
            self.items[index], self.items[smallest_known_index] = self.items[smallest_known_index], self.items[index]
            
            # update map
            self.map[self.items[index].value] = index
            self.map[self.items[smallest_known_index].value] = smallest_known_index

            # recursive call
            self.sink_down(smallest_known_index)

    def build_heap(self):
        for i in range(self.length // 2 - 1, -1, -1):
            self.sink_down(i) 

    def insert(self, node):
        self.items.append(node)
        self.map[node.value] = self.length
        self.length += 1
        self.swim_up(self.length - 1)

    def swim_up(self, index):
        while index > 0 and self.items[index].key < self.items[self.find_parent_index(index)].key:
            parent_index = self.find_parent_index(index)
            self.items[index], self.items[parent_index] = self.items[parent_index], self.items[index]
            #update map
            self.map[self.items[index].value] = index
            self.map[self.items[parent_index].value] = parent_index
            index = parent_index

    def extract_min(self):
        if self.length == 0:
            return None
        #exchange
        self.items[0], self.items[self.length - 1] = self.items[self.length - 1], self.items[0]
        #update map
        self.map[self.items[self.length - 1].value] = self.length - 1
        self.map[self.items[0].value] = 0

        min_node = self.items.pop()
        self.length -= 1
        self.map.pop(min_node.value, None)
        if self.length > 0:
            self.sink_down(0)
        return min_node

    def decrease_key(self, value, new_key):
        if value in self.map and new_key < self.items[self.map[value]].key:
            index = self.map[value]
            self.items[index].key = new_key
            self.swim_up(index)

    def is_empty(self):
        return self.length == 0

class Item:
    def __init__(self, value, key):
        self.key = key
        self.value = value
    
    def __str__(self):
        return "(" + str(self.key) + "," + str(self.value) + ")"

class WeightedGraph:
    def __init__(self, nodes):
        self.graph = [[] for _ in range(nodes)]
        self.weights = {}

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def get_weights(self, node1, node2):
        return self.weights.get((node1, node2))

    def get_neighbors(self, node):
        return self.graph[node]

    def get_number_of_nodes(self):
        return len(self.graph)

In [ ]:
# PART 1.1
# DIJKSTRA'S ALGORITHM

def dijkstra(graph, source, k):
    visited = {i: False for i in range(graph.get_number_of_nodes())}
    distance = {i: float("inf") for i in range(graph.get_number_of_nodes())}
    path = {i: [] for i in range(graph.get_number_of_nodes())}
    relaxation_count = {i: 0 for i in range(graph.get_number_of_nodes())}
    
    Q = MinHeap([Item(i, float("inf")) for i in range(graph.get_number_of_nodes())])
    distance[source] = 0
    Q.decrease_key(source, 0)
    path[source] = [source]

    while not Q.is_empty():
        current_item = Q.extract_min()
        current_node = current_item.value
        visited[current_node] = True

        for neighbor in graph.get_neighbors(current_node):
            if visited[neighbor] or relaxation_count[neighbor] >= k:
                continue

            edge_weight = graph.get_weights(current_node, neighbor)
            new_distance = distance[current_node] + edge_weight

            if new_distance < distance[neighbor]:
                distance[neighbor] = new_distance
                path[neighbor] = path[current_node] + [neighbor]
                Q.decrease_key(neighbor, new_distance)
                relaxation_count[neighbor] += 1

    return distance, path

# Example usage:
# g = WeightedGraph(5)
# g.add_edge(0, 1, 10)
# g.add_edge(0, 2, 3)
# g.add_edge(1, 2, 1)
# g.add_edge(1, 3, 2)
# g.add_edge(2, 1, 4)
# g.add_edge(2, 3, 8)
# g.add_edge(2, 4, 2)
# g.add_edge(3, 4, 7)
# g.add_edge(4, 3, 9)
# print(dijkstra(g, 0, 3))

In [ ]:
#PART 1.3
#EXPERIMENT TO ANALYZE PERFORMANCE
